# MuchLearningSuchWow - LSTM - Preprocessing

This notebook contains the code we used to preprocess the data. The preprocessing code is based primarily on [this kernel](https://www.kaggle.com/bountyhunters/baseline-lstm-with-keras-0-7).

### Imports & Data Paths

In [1]:
import numpy as np
import pandas as pd
import pickle

from sklearn.preprocessing import MinMaxScaler

In [2]:
inputPath = "input/m5-forecasting-accuracy/"
outputPath = "output/"

### Constants

In [3]:
startDay = 1000 # Number of days at start of data that will be ignored during training

### Loading Data

In [4]:
with open(outputPath + "/downcasted_sales_train_evaluation.pkl", "rb") as f:
    df_sales = pickle.load(f)
df_calendar = pd.read_csv(inputPath + "/calendar.csv")

In [5]:
df_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


### Preprocessing

In [6]:
# Transpose the training data so that rows represent days and columns represent items
df_sales = df_sales.T
print(df_sales.shape)
df_sales.head(11)

(1947, 30490)


,0,1,2,3,4,5,6,7,8,9,...,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
id,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_010_CA_1_evaluation,...,FOODS_3_818_WI_3_evaluation,FOODS_3_819_WI_3_evaluation,FOODS_3_820_WI_3_evaluation,FOODS_3_821_WI_3_evaluation,FOODS_3_822_WI_3_evaluation,FOODS_3_823_WI_3_evaluation,FOODS_3_824_WI_3_evaluation,FOODS_3_825_WI_3_evaluation,FOODS_3_826_WI_3_evaluation,FOODS_3_827_WI_3_evaluation
item_id,HOBBIES_1_001,HOBBIES_1_002,HOBBIES_1_003,HOBBIES_1_004,HOBBIES_1_005,HOBBIES_1_006,HOBBIES_1_007,HOBBIES_1_008,HOBBIES_1_009,HOBBIES_1_010,...,FOODS_3_818,FOODS_3_819,FOODS_3_820,FOODS_3_821,FOODS_3_822,FOODS_3_823,FOODS_3_824,FOODS_3_825,FOODS_3_826,FOODS_3_827
dept_id,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,...,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3
cat_id,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,...,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS
store_id,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,...,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3
state_id,CA,CA,CA,CA,CA,CA,CA,CA,CA,CA,...,WI,WI,WI,WI,WI,WI,WI,WI,WI,WI
d_1,0,0,0,0,0,0,0,12,2,0,...,0,14,1,0,4,0,0,0,0,0
d_2,0,0,0,0,0,0,0,15,0,0,...,0,11,1,0,4,0,0,6,0,0
d_3,0,0,0,0,0,0,0,0,7,1,...,0,5,1,0,2,2,0,0,0,0
d_4,0,0,0,0,0,0,0,0,3,0,...,0,6,1,0,5,2,0,2,0,0


In [7]:
# Remove id, item_id, dept_id, cat_id, store_id and state_id rows, as well as the first "startDay" days
item_data = df_sales[:6] # Save rows for possible future use
df_sales = df_sales[6 + startDay:]
print(df_sales.shape)
df_sales.head()

(941, 30490)


,0,1,2,3,4,5,6,7,8,9,...,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
d_1001,2,0,0,0,1,1,1,10,0,0,...,1,1,0,1,2,0,0,0,0,0
d_1002,2,1,0,8,1,0,0,3,4,0,...,3,4,2,0,5,0,0,0,0,0
d_1003,0,0,0,6,0,0,1,20,2,0,...,2,1,4,0,2,0,0,0,0,0
d_1004,1,0,0,3,0,6,0,9,0,0,...,1,0,1,0,6,0,0,0,1,0
d_1005,1,0,0,2,0,2,0,3,0,0,...,0,3,1,0,3,0,0,0,5,0


### Additional Features 

##### One day before event

In [8]:
# Create dataframe with zeros for 1969 days in the calendar
days_before_event = pd.DataFrame(np.zeros((1969,1)))

In [9]:
# Assign "1" to days before which there is an event_name_1 (event_name_2 never occurs without event_name_1, so it is redundant)
for x,y in df_calendar.iterrows():
    if((pd.isnull(df_calendar["event_name_1"][x])) == False and x != 0):
        days_before_event[0][x-1] = 1

###### Weekend

In [10]:
# Create dataframe with zeros for 1969 days in the calendar
weekend = pd.DataFrame(np.zeros((1969,1)))

In [11]:
# Assign "1" to saturdays and sundays
for x,y in df_calendar.iterrows():
    if(df_calendar["weekday"][x] == "Saturday" or df_calendar["weekday"][x] == "Sunday"):
        weekend[0][x] = 1

##### Months

In [12]:
# Create dataframe with zeros for 1969 days in the calendar, with one column for each month
months_one_hot = pd.DataFrame(np.zeros((1969,12)))

In [13]:
# Assign "1" in the correct column for each day
for x,y in df_calendar.iterrows():
    months_one_hot[df_calendar["month"][x] - 1][x] = 1

##### Combining features 

In [14]:
additional_features = pd.DataFrame(np.concatenate([days_before_event,weekend,months_one_hot], axis = 1))
print(additional_features.shape)

(1969, 14)


In [15]:
del df_calendar

##### Splitting and Merging

In [16]:
# Split additional_features into training, validation and evaluation parts
additional_features_train_valid = additional_features[startDay:1941]
additional_features_valid = additional_features[1913:1941]
additional_features_eval  = additional_features[1941:1969]
del additional_features

In [17]:
# Change column names of the train + validation part to match training data and match indices
additional_features_train_valid.columns = ["one_day_before_event", "weekend", "january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
additional_features_train_valid.index = df_sales.index
additional_features_train_valid.head(10)

,one_day_before_event,weekend,january,february,march,april,may,june,july,august,september,october,november,december
d_1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
d_1002,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
d_1003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
d_1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
d_1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
d_1006,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
d_1007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
d_1008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
d_1009,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
d_1010,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [18]:
# Add additional_features_train_valid to the sales data to obtain the training & validation set
df_train_valid = pd.concat([df_sales, additional_features_train_valid], axis = 1)
df_train_valid.columns

Index([          0,           1,           2,           3,           4,
                 5,           6,           7,           8,           9,
       ...
           'march',     'april',       'may',      'june',      'july',
          'august', 'september',   'october',  'november',  'december'],
      dtype='object', length=30504)

### Feature Scaling

In [19]:
# Split training set, for scaling
df_train = df_train_valid[:-28]
print(df_train.shape)

(913, 30504)


In [20]:
# Create a min-max scaler that scales the training features to range 0-1
scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(df_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

### Saving Results

In [21]:
# Save the validation and evaluation parts of the additional features
with open(outputPath + "/additional_features_testing.pkl", "wb") as f:
    pickle.dump((additional_features_valid, additional_features_eval), f)

In [22]:
# Save the scaler
with open(outputPath + "/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

In [23]:
# Save the preprocessed (train + validation) data
with open(outputPath + "/preprocessed_train_valid_data.pkl", "wb") as f:
    pickle.dump(df_train_valid, f)